In [4]:
from coolest.template.standard import COOLEST
from coolest.template.json import JSONSerializer
import os
from coolest.template.classes.noise import NoiseMap
from coolest.template.classes.psf import PixelatedPSF
from coolest.template.classes.instrument import Instrument
from coolest.template.classes.observation import Observation
from coolest.template.classes.coordinates import CoordinatesOrigin
from coolest.template.classes.cosmology import Cosmology
from coolest.template.classes.galaxy import Galaxy
from coolest.template.classes.mass_field import MassField
from coolest.template.classes.lensing_entity_list import LensingEntityList
from coolest.template.classes.mass_light_model import MassModel, LightModel
from coolest.template.classes.grid import PixelatedRegularGrid

## Coolest Template Generator
In this notebook we generate coolest template (.json file) needed to update using 

lenstronomy kwargs of Lens class in slsim. The template generated by this notebook 

should be supplied to update_coolest_from_lenstronomy_slsim() function. To run this 

notebook one need to download test_image.fits from slsim/tests/TestData and put this 

image to your desired path.

In [11]:
# Setup cosmology
cosmology = Cosmology(H0=78.0, Om0=0.3)

# Create a source galaxy and point source (if needed) at a desired redshift.
source_1 = Galaxy("a source galaxy", 2.0, light_model=LightModel("Sersic"))
source_2 = Galaxy("a source galaxy", 2.0, light_model=LightModel("LensedPS"))

# Create a lens galaxy
lens_1 = Galaxy(
    "a lens galaxy", 0.5, light_model=LightModel("Sersic"), mass_model=MassModel("PEMD")
)
ext_shear = MassField(
    "external shear", lens_1.redshift, mass_model=MassModel("ExternalShear")
)


# Put them in a list, which will also create unique IDs for each profile. If you need
# more source galaxies, you can add here.
entity_list = LensingEntityList(lens_1, ext_shear, source_1, source_2)

# Define the origin of the coordinates system
origin = CoordinatesOrigin("00h0m0s", "0d0m0s")

# Provide data file""", download this test_image.fits from slsim/tests/TestData or
# choose your own image and put it to your own folder.
obs_pixels = PixelatedRegularGrid(
    "test_image.fits", check_fits_file=True  # absolute path just for unit tests
)


# Select the type of noise
# noise = InstrumentalNoise()
noise = NoiseMap(noise_map = obs_pixels)

observation = Observation(pixels=obs_pixels, noise=noise)

# Defines the instrument
# psf = PixelatedPSF(PixelatedRegularGrid('test/test_psf.fits',   # absolute path just for unit tests
# check_fits_file=self.check_files))
psf = PixelatedPSF(obs_pixels)

instrument = Instrument(
    0.08, name="some instrument", readout_noise=4, band="F160W", psf=psf  # pixel size
)

# Master object for the standard
coolest = COOLEST("MOCK", origin, entity_list, observation, instrument, cosmology)

# export as JSON file. Give a path where you have stored test_image.fits.
template_path = os.path.join(
    "/Users/narayankhadka/Desktop/strong_lensing", "coolest_template"
)
serializer = JSONSerializer(template_path, obj=coolest, check_external_files=True)
serializer.dump_simple()